In [ ]:
import os
import pandas as pd
import zipfile
class MP:
    def __init__(self):
        self.dict_from_params,self.dict_to_params = self.from_to_div()#делим словарь параметров на параметры from и to
        listdir = os.listdir(os.getcwd()+'\\'+self.name)#список нужных файлов
        files = self.multipleer([[self.name+'\\'+i for i in listdir if word in i.lower()] for word in self.key_words])#группируем файлы
        print(files)
        self.dict_df ={}#словарь Лист:Датафрейм
        for i in range(len(list(self.dict_to_params))):
            self.dict_df[list(self.dict_to_params)[i]]=self.read(files[i],list(self.dict_to_params)[i])
        self.editing_data()
        
    def from_to_div(self):
        f,to = {},{}
        for key in self.from_to:
            if 'from' in self.from_to[key]:
                f[key] = self.from_to[key]['from']
            if 'to' in self.from_to[key]:
                to[key] = self.from_to[key]['to']
        return (f,to)   
        
    def multipleer(self,files):#множитель файлов для считывания(менял только в яндексе)
        return files
        
    def from_params(self,sheet_name=False,skiprows=0,begin=0):
        return {'sheet_name':sheet_name,'skiprows':skiprows,'begin':begin}
        
        
    def to_params(self,index=False,startcol=0,header=True, startrow=0):#параметры для переноса фрейма
        return {'index':index,'startcol':startcol,'header':header,'startrow':startrow}
        
    def read(self,i_files,name):#функция чтения  таблицы(таблиц)
        read_file = self.read_zip if 'zip' in i_files[0] else pd.read_excel
        if name in self.dict_from_params:#когда есть параметры на чтение во всех парметрах
            params = self.dict_from_params[name]
            if params['sheet_name']:
                i_s = [read_file(i_file,sheet_name = params['sheet_name'],skiprows = params['skiprows'])[params['begin']:] for i_file in i_files]
            else:
                i_s = [read_file(i_file,skiprows = params['skiprows'])[params['begin']:] for i_file in i_files]
        else:
            i_s = [read_file(i_file) for i_file in i_files]
        if len(i_s)==1:
            return i_s[0]
        return pd.concat(i_s, ignore_index=True)
        
    def read_zip(self, f_zip,sheet_name=0,skiprows=0):#чтение zip      
        with zipfile.ZipFile(f_zip) as z:
            v = z.namelist()[0]
            with z.open(v) as f:
                train = pd.read_excel(f, sheet_name=sheet_name, skiprows=skiprows)
        return train
        
    def editing_data(self):#редактирование считанных данных
        pass
        
    def write(self,name):#функция переноса таблиц
        with pd.ExcelWriter(name, mode='a', if_sheet_exists='overlay') as writer:
            for key in list(self.dict_df):
                self.dict_df[key].to_excel(writer, sheet_name=key,
                                           index=self.dict_to_params[key]['index'],
                                           startcol=self.dict_to_params[key]['startcol'],
                                           header=self.dict_to_params[key]['header'],
                                          startrow=self.dict_to_params[key]['startrow'])

        
class Ozon(MP):
    def __init__(self):
        self.name = 'Озон'
        self.key_words = ['заказ','начис','себ']
    
        self.from_to =  {'Заказы':{'to':self.to_params(startcol=1)},
                    'Начисления и компенсации по тов':{'from':self.from_params(skiprows=1),'to':self.to_params(startcol=1,header=False,startrow=1)},
                    'Себестоимость':{'to':self.to_params()}}
        super().__init__()
    def editing_data(self):
        art_old = ['ДO013/1','ДO013/11','ДO013/13','ДO013/2','ДO013/20','ДO013/22','ДO013/3','ДO013/38','ДO013/6','ДO013/7','ДO013/96','ДO013/97','ДO013/16']
        art_now = ['ДО013/1','ДО013/11','ДО013/13','ДО013/2','ДО013/20','ДО013/22','ДО013/3','ДО013/38','ДО013/6','ДО013/7','ДО013/96','ДО013/97','ДО013/16']
        self.dict_df['Себестоимость'] = self.dict_df['Себестоимость'].replace(art_old,art_now)
        self.dict_df['Начисления и компенсации по тов'] = self.dict_df['Начисления и компенсации по тов'].drop(columns='ID начисления')
        
    def write(self,name = 'Озон шаблон.xlsx'):
        super().write(name)
        print('Запись в Озон завершена')

   
class JM(MP):
    def __init__(self):
        self.name = 'ЯМ'
        self.key_words = ['остат','заказ','стоимости','себ']
    
        self.from_to = {'Остатки по складам':{'from':self.from_params(skiprows=3,begin=1),'to':self.to_params(header=False, startcol=1,startrow=7)},
                    'фин отчет по заказам':{'from':self.from_params(sheet_name='Транзакции по заказам и товарам',skiprows=8,begin=1),'to':self.to_params(header=False, startcol=4,startrow=5)},
                    'Размещение товаров на витрине':{'from':self.from_params(sheet_name='Размещение товаров на витрине',skiprows=4,begin=2),'to':self.to_params(index=False,header=False, startcol=1,startrow=6)},
                    'Складская обработка':{'from':self.from_params(sheet_name='Складская обработка',skiprows=1),'to':self.to_params(header=False, startcol=1,startrow=3)},
                    'Программа лояльности и отзывы':{'from':self.from_params(sheet_name='Программа лояльности и отзывы',skiprows=1),'to':self.to_params(header=False, startcol=1,startrow=3)},
                    'Буст продаж':{'from':self.from_params(sheet_name='Буст продаж, оплата за продажи',skiprows = 1),'to':self.to_params(header=False, startcol=1,startrow=3)},
                    'Доставка покупателю':{'from':self.from_params(sheet_name='Доставка покупателю',skiprows=1),'to':self.to_params(header=False, startcol=1,startrow=3)},
                    'Экспресс-доставка покупателю':{'from':self.from_params(sheet_name='Экспресс-доставка покупателю',skiprows=1),'to':self.to_params(header=False, startcol=1,startrow=3)},
                    'Приём платежа':{'from':self.from_params(sheet_name='Приём платежа',skiprows=4),'to':self.to_params(index=False,header=False, startcol=1,startrow=6)},
                    'Перевод платежа':{'from':self.from_params(sheet_name='Перевод платежа',skiprows=1),'to':self.to_params(header=False, startcol=1,startrow=3)},
                    'Обработка заказа':{'from':self.from_params(sheet_name='Обработка заказов на складе',skiprows=1),'to':self.to_params(header=False, startcol=1,startrow=3)},
                    'себес':{'to':self.to_params()}}
        super().__init__()
        
    def multipleer(self,files):
        return files[:-2]+9*[files[-2]]+[files[-1]]
        
    def write(self,name = 'ЯМ шаблон.xlsx'):
        super().write(name)
        print('Запись в Яндекс маркет завершена')

class WB(MP):
    def __init__(self):
        self.name = 'ВБ'
        self.key_words = ['воронка продаж','детал','продажи','хран','остат','оцен','себ']
        
        self.from_to = {'% выкупа':{'from':self.from_params(sheet_name='Товары',skiprows=1),'to':self.to_params()},
                              'Детализация отчётов':{'to':self.to_params()},
                              'Продажи':{'from':self.from_params(skiprows=1),'to':self.to_params()},
                              'Сумма по хранению':{'from':self.from_params(sheet_name='Отчёт хранение'), 'to':self.to_params()},
                              'Остатки по складам':{'to':self.to_params()},
                              'Рейтинг по отзывам':{'to':self.to_params()},
                              'Себестоимость':{'to':self.to_params()}}
        super().__init__()
    
    def editing_data(self):
        self.dict_df['Сумма по хранению']=self.dict_df['Сумма по хранению'][['Артикул продавца','Сумма хранения, руб']].groupby('Артикул продавца').sum().reset_index()
        self.dict_df['Остатки по складам']=self.dict_df['Остатки по складам'][['Артикул продавца','Всего находится на складах']].fillna(0).groupby('Артикул продавца').sum().reset_index()
        self.dict_df['Рейтинг по отзывам']['Рейтинг отзывов'] = self.dict_df['Рейтинг по отзывам']['Рейтинг отзывов'].str.split().str[0].str.replace('-','0').astype(float)
        voronka_drop = ['Удаленный товар','Рейтинг по отзывам','Рейтинг карточки', 'Сумма остатков на складах, ₽','Среднее время доставки','Среднее время доставки (предыдущий период)',
               'Локальные заказы, %','Локальные заказы, % (предыдущий период)','Доля карточки в выручке','Доля карточки в выручке (предыдущий период)']
        self.dict_df['рейтинг WB'] = self.dict_df['% выкупа'][['Артикул продавца','Рейтинг карточки']]
        self.dict_df['Индекс локалицации'] = self.dict_df['% выкупа'][['Артикул продавца','Локальные заказы, %']]
        self.dict_df['Индекс локалицации']['Локальные заказы, %'] = self.dict_df['Индекс локалицации']['Локальные заказы, %']/100
        self.voronka = self.dict_df['% выкупа']
        self.dict_df['% выкупа'] = self.voronka[['Артикул продавца','Процент выкупа']]
        self.voronka.drop(voronka_drop, axis=1, inplace=True)
        self.dict_to_params['рейтинг WB'] = self.to_params()
        self.dict_to_params['Индекс локалицации'] = self.to_params()
        
    def write(self,name = 'WB шаблон.xlsx'):
        super().write(name)
        print('Запись в WB завершена')
        with pd.ExcelWriter('Воронка WB шаблон.xlsx', mode='a', if_sheet_exists='overlay') as writer:
            self.voronka.to_excel(writer, sheet_name='Тех отчет',index=False,header=False,startrow=1)
        print('Запись в воронку WB завершена')



In [ ]:
j = JM()
j.write()

In [ ]:
o = Ozon()
o.write()

In [ ]:
w = WB()
w.write()